<a target="_blank" href="https://colab.research.google.com/github/Jasonchen0317/CSGY-6613-Assignment/blob/main/assignment-3/assignment-3.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
from pytube import YouTube
import os

In [2]:
def download_video(video_list):
    for i in range(len(video_list)):
        #get link
        link = video_list[i]
        yt = YouTube(link)
        yt.bypass_age_gate()
        #Download video
        filename = "%s.mp4" % str(i)
        yt.streams.filter(res="720p").first().download(filename=filename, output_path='video')

In [ ]:
if not os.path.exists('video'):
   os.makedirs('video')
   
video_list = [
    'https://www.youtube.com/watch?v=DhmZ6W1UAv4',
    'https://www.youtube.com/watch?v=YrydHPwRelI'
]

In [ ]:
download_video(video_list)